In [1]:
import numpy as np

In [2]:
import ipyvolume as ipv

In [3]:
import matplotlib.pyplot as plt

In [4]:
%matplotlib widget

In [5]:
import xarray as xr
import h5py

In [6]:
def hdf5_to_xarray(h5_data):
    """Convert hdf5 dataset to xarray dataset."""
    data = h5_data[:]
    coords = [(dim.label, dim[0][:]) for dim in h5_data.dims]
    xr_data = xr.DataArray(data, coords=[coords[0], coords[3], coords[2], coords[1]])
    return xr_data

In [7]:
def get_data(setnum, task, skip=1):
    # Load file
    filename = "/Users/kburns/Desktop/rbc_s%i.h5" %setnum
    file = h5py.File(filename, 'r')
    # Get data
    #return file['tasks'][task]
    xr = hdf5_to_xarray(file['tasks'][task])
    return xr

In [17]:
def extrap_full_sphere(X):
    # Extrapolate in theta
    X_0 = X.interp(theta=0, kwargs={'fill_value': 'extrapolate'})
    X_pi = X.interp(theta=np.pi, kwargs={'fill_value': 'extrapolate'})
    X = xr.concat([X_pi, X, X_0], dim='theta')
    # Extrapolate in r
    X_0 = X.interp(r=0, kwargs={'fill_value': 'extrapolate'})
    X = xr.concat([X_0, X], dim='r')
    return X

In [18]:
T = extrap_full_sphere(get_data(1, 'T'))
T.coords['z'] = T.coords['r'] * np.cos(T.coords['theta'])
T.coords['x'] = T.coords['r'] * np.sin(T.coords['theta']) * np.cos(T.coords['phi'])

In [19]:
T

<xarray.DataArray (t: 34, phi: 384, theta: 194, r: 193)>
array([[[[ 8.030939e-05, ..., -4.185452e-05],
         ...,
         [-2.877264e-05, ...,  4.219132e-04]],

        ...,

        [[ 8.030902e-05, ..., -3.816457e-05],
         ...,
         [-2.877235e-05, ...,  4.187921e-04]]],


       ...,


       [[[-5.485435e-02, ...,  8.610606e-08],
         ...,
         [-5.478079e-02, ..., -8.713279e-08]],

        ...,

        [[-5.485435e-02, ...,  8.603595e-08],
         ...,
         [-5.478080e-02, ..., -8.702599e-08]]]])
Coordinates:
  * t        (t) float64 0.0 0.001581 0.003162 ... 0.04857 0.05013 0.0517
  * phi      (phi) float64 0.0 0.01636 0.03272 0.04909 ... 6.234 6.25 6.267
  * theta    (theta) float64 3.142 3.129 3.113 3.097 ... 0.02868 0.01249 0.0
  * r        (r) float64 0.0 0.008149 0.0163 0.02445 ... 0.9997 0.9999 1.0
    z        (r, theta) float64 -0.0 -0.0 -0.0 -0.0 ... 0.999 0.9996 0.9999 1.0
    x        (r, theta, phi) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.

In [38]:
dpi = 200
s = 1 / np.sqrt(3)
plt.figure(figsize=(6,6))
background = T.coords['z']

for i in range(34):
    dTi = T.isel(t=i) - background
    plt.clf()
    vmax = np.max(np.abs(dTi))
    dTi.isel(phi=0).plot(x='x', y='z', add_colorbar=False, vmin=-vmax, vmax=vmax, cmap='RdBu')
    dTi.isel(phi=192).plot(x='x', y='z', add_colorbar=False, vmin=-vmax, vmax=vmax, cmap='RdBu')    
    plt.title('')
    plt.axis('equal')
    plt.axis('off')
    plt.savefig('sphere_%03i.png' %i, dpi=dpi)
    line, = plt.plot([s,s,-s,-s,s], [-s,s,s,-s,-s], '--k', lw=1)
    plt.savefig('spherebox_%03i.png' %i, dpi=dpi)
    line.set_visible(False)
    plt.xlim(-s, s)
    plt.ylim(-s, s)
    plt.savefig('box_%03i.png' %i, dpi=dpi)

/Users/kburns/Software/miniconda3/envs/dedalus-openblas/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

###### 

In [39]:
T.shape

(34, 384, 194, 193)

In [8]:
def volbot(setnum, task, skip=1):
    data = get_data(setnum, task, skip=skip)
    # Correct z offset
    data = np.roll(data, 64//skip, axis=2)
    # Renormalize
    data /= np.sqrt(np.mean(data**2))
    data[data < -2] = -2
    data[data > 2] = 2
    # Volume rendering
    ipv.figure(width=1000, height=1000)
    ipv.style.axes_off()
    vol = ipv.volshow(data, level=[0.25, 0.25, 0.75], opacity=[0.1, 0.02, 0.1], level_width=0.05)
    vol.brightness = 2.0
    ipv.pylab.view(azimuth=90, elevation=30, distance=2)
    ipv.show()

In [ ]:
volbot(7, 'ωz', skip=4)